In [3]:
import pandas as pd

df = pd.read_csv("/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/6 - limpeza e pre processamento/tecnClean.csv")


In [4]:
columns_to_drop = [
   'grupo10',
   'grupo11',
   'grupo12',
   'grupo13',
   'grupo14',
   'grupo15',
   'grupo16',
   'grupo17',
   'grupo18',
   'grupo19',
   'grupo1a',
   'grupo1b',
   'grupo1c',
   'grupo2',
   'grupo20',
   'grupo21',
   'grupo22',
   'grupo3',
   'grupo4',
   'grupo5',
   'grupo6',
   'grupo7',
   'grupo8',
   'grupo9a',
   'grupo9b',
   'h10b3_sindrome_fibrose',
   'h10b4_sindrome_fenilcetonuria',
   's09_altura_feita_corrigida',
   'vd_dummy_gravida'
]

# Remove the specified columns from the dataframe
DS_Near_zero = df.drop(columns=columns_to_drop)

# Save the new dataset to the specified path
DS_Near_zero.to_csv("/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/6 - limpeza e pre processamento/DS_Near_zero.csv", index=False)

print(f"New dataset created with {DS_Near_zero.shape[0]} rows and {DS_Near_zero.shape[1]} columns")
print("Dataset saved successfully!")

New dataset created with 1960 rows and 294 columns
Dataset saved successfully!


In [8]:
# Load the DS_Near_zero dataset
df_near_zero = pd.read_csv("/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/6 - limpeza e pre processamento/DS_Near_zero.csv")

# Load the dictionary Excel file
df_dictionary = pd.read_excel("/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/6 - limpeza e pre processamento/4-Dicionario-ENANI-2019 (1).xlsx")

# Get the column names from DS_Near_zero
columns_in_dataset = df_near_zero.columns.tolist()

# Cross-reference with the dictionary to get descriptions
matched_columns = df_dictionary[df_dictionary['variavel'].isin(columns_in_dataset)].drop_duplicates(subset=['variavel'])

print(f"Total columns in DS_Near_zero: {len(columns_in_dataset)}")
print(f"Matched columns with dictionary: {len(matched_columns)}")
print("\nColumns with descriptions and value distributions:")
print("="*80)

for _, row in matched_columns.iterrows():
    col = row['variavel']
    description = row['descricao']
    
    print(f"\nVariable: {col}")
    print(f"Description: {description}")
    
    # Get value counts and percentages for this column
    if col in df_near_zero.columns:
        value_counts = df_near_zero[col].value_counts(dropna=False)
        percentages = df_near_zero[col].value_counts(normalize=True, dropna=False) * 100
        
        print(f"Total responses: {len(df_near_zero[col])}")
        print("Value distribution:")
        
        for value, count in value_counts.items():
            percentage = percentages[value]
            print(f"  • {value}: {count} ({percentage:.2f}%)")
    
    print("-" * 60)

# Show unmatched columns
unmatched_columns = [col for col in columns_in_dataset if col not in df_dictionary['variavel'].values]
if unmatched_columns:
    print(f"\nColumns without dictionary match ({len(unmatched_columns)}):")
    for col in unmatched_columns:
        print(f"\nVariable: {col}")
        print("Description: No description available")
        
        # Get value counts and percentages for unmatched columns too
        value_counts = df_near_zero[col].value_counts(dropna=False)
        percentages = df_near_zero[col].value_counts(normalize=True, dropna=False) * 100
        
        print(f"Total responses: {len(df_near_zero[col])}")
        print("Value distribution:")
        
        for value, count in value_counts.items():
            percentage = percentages[value]
            print(f"  • {value}: {count} ({percentage:.2f}%)")
        
        print("-" * 60)

/opt/anaconda3/envs/pycaret_env/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Total columns in DS_Near_zero: 294
Matched columns with dictionary: 293

Columns with descriptions and value distributions:

Variable: a00_regiao
Description: a00_regiao - Macrorregião 
Total responses: 1960
Value distribution:
  • Centro-Oeste: 463 (23.62%)
  • Sul: 461 (23.52%)
  • Sudeste: 392 (20.00%)
  • Norte: 330 (16.84%)
  • Nordeste: 314 (16.02%)
------------------------------------------------------------

Variable: a11_situacao
Description: a11_situacao - Situação do setor do domicílio
Total responses: 1960
Value distribution:
  • Urbano: 1922 (98.06%)
  • Rural: 38 (1.94%)
------------------------------------------------------------

Variable: b02_sexo
Description: b02 - Sexo da criança 
Total responses: 1960
Value distribution:
  • Feminino: 1005 (51.28%)
  • Masculino: 955 (48.72%)
------------------------------------------------------------

Variable: b03_relacao
Description: b03_relacao - Relação da criança com o responsável pelo domicílio
Total responses: 1960
Value di

In [9]:
pesos_amostrais = [    # Pesos amostrais (todos)
    'peso_crianca',
    'peso_crianca_calib',
    'peso_crianca_setor',
    'peso_crianca_y_10', 'peso_crianca_y_11', 'peso_crianca_y_12',
    'peso_crianca_y_13', 'peso_crianca_y_14', 'peso_crianca_y_15',
    'peso_crianca_y_16', 'peso_crianca_y_17', 'peso_crianca_y_18',
    'peso_crianca_y_19', 'peso_crianca_y_1a', 'peso_crianca_y_1b',
    'peso_crianca_y_1c', 'peso_crianca_y_2', 'peso_crianca_y_20',
    'peso_crianca_y_21', 'peso_crianca_y_22', 'peso_crianca_y_3',
    'peso_crianca_y_4', 'peso_crianca_y_5', 'peso_crianca_y_6',
    'peso_crianca_y_7', 'peso_crianca_y_8', 'peso_crianca_y_9a',
    'peso_crianca_y_9b']

data_leakage = [
        # Data leakage
    'k11_amamentou',
    'k14_ainda', 
    'k18_somente',
]

administrativas = [
        'p01_respondente',
    'm02_quem_cozinha',
    'm11_quem_decide',
    'm12_despesas',
]

# Combine all column lists to create a comprehensive list of columns to remove
columns_to_remove = pesos_amostrais + data_leakage + administrativas

print(f"Total columns to remove: {len(columns_to_remove)}")
print(f"Columns to remove: {columns_to_remove}")

# Remove the specified columns from the DS_Near_zero dataset
df_final = df_near_zero.drop(columns=columns_to_remove)

# Save the final cleaned dataset
df_final.to_csv("/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/6 - limpeza e pre processamento/DS_Final_Cleaned.csv", index=False)

print(f"\nFinal dataset created with {df_final.shape[0]} rows and {df_final.shape[1]} columns")
print(f"Removed {len(columns_to_remove)} columns")
print("Final cleaned dataset saved successfully!")

Total columns to remove: 35
Columns to remove: ['peso_crianca', 'peso_crianca_calib', 'peso_crianca_setor', 'peso_crianca_y_10', 'peso_crianca_y_11', 'peso_crianca_y_12', 'peso_crianca_y_13', 'peso_crianca_y_14', 'peso_crianca_y_15', 'peso_crianca_y_16', 'peso_crianca_y_17', 'peso_crianca_y_18', 'peso_crianca_y_19', 'peso_crianca_y_1a', 'peso_crianca_y_1b', 'peso_crianca_y_1c', 'peso_crianca_y_2', 'peso_crianca_y_20', 'peso_crianca_y_21', 'peso_crianca_y_22', 'peso_crianca_y_3', 'peso_crianca_y_4', 'peso_crianca_y_5', 'peso_crianca_y_6', 'peso_crianca_y_7', 'peso_crianca_y_8', 'peso_crianca_y_9a', 'peso_crianca_y_9b', 'k11_amamentou', 'k14_ainda', 'k18_somente', 'p01_respondente', 'm02_quem_cozinha', 'm11_quem_decide', 'm12_despesas']

Final dataset created with 1960 rows and 259 columns
Removed 35 columns
Final cleaned dataset saved successfully!


In [10]:
# Load the DS_Final_clean dataset
DS_Final_clean = pd.read_csv("/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/6 - limpeza e pre processamento/DS_Final_Cleaned.csv")

# Load the dictionary Excel file
df_dictionary = pd.read_excel("/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/6 - limpeza e pre processamento/4-Dicionario-ENANI-2019 (1).xlsx")

# Get the column names from DS_Final_clean
columns_in_dataset = DS_Final_clean.columns.tolist()

# Cross-reference with the dictionary to get descriptions
matched_columns = df_dictionary[df_dictionary['variavel'].isin(columns_in_dataset)].drop_duplicates(subset=['variavel'])

print(f"Total columns in DS_Final_clean: {len(columns_in_dataset)}")
print(f"Matched columns with dictionary: {len(matched_columns)}")
print("\nColumns with descriptions and value distributions:")
print("="*80)

for _, row in matched_columns.iterrows():
    col = row['variavel']
    description = row['descricao']
    
    print(f"\nVariable: {col}")
    print(f"Description: {description}")
    
    # Get value counts and percentages for this column
    if col in df_near_zero.columns:
        value_counts = df_near_zero[col].value_counts(dropna=False)
        percentages = df_near_zero[col].value_counts(normalize=True, dropna=False) * 100
        
        print(f"Total responses: {len(df_near_zero[col])}")
        print("Value distribution:")
        
        for value, count in value_counts.items():
            percentage = percentages[value]
            print(f"  • {value}: {count} ({percentage:.2f}%)")
    
    print("-" * 60)

# Show unmatched columns
unmatched_columns = [col for col in columns_in_dataset if col not in df_dictionary['variavel'].values]
if unmatched_columns:
    print(f"\nColumns without dictionary match ({len(unmatched_columns)}):")
    for col in unmatched_columns:
        print(f"\nVariable: {col}")
        print("Description: No description available")
        
        # Get value counts and percentages for unmatched columns too
        value_counts = df_near_zero[col].value_counts(dropna=False)
        percentages = df_near_zero[col].value_counts(normalize=True, dropna=False) * 100
        
        print(f"Total responses: {len(df_near_zero[col])}")
        print("Value distribution:")
        
        for value, count in value_counts.items():
            percentage = percentages[value]
            print(f"  • {value}: {count} ({percentage:.2f}%)")
        
        print("-" * 60)

Total columns in DS_Final_clean: 259
Matched columns with dictionary: 258

Columns with descriptions and value distributions:

Variable: a00_regiao
Description: a00_regiao - Macrorregião 
Total responses: 1960
Value distribution:
  • Centro-Oeste: 463 (23.62%)
  • Sul: 461 (23.52%)
  • Sudeste: 392 (20.00%)
  • Norte: 330 (16.84%)
  • Nordeste: 314 (16.02%)
------------------------------------------------------------

Variable: a11_situacao
Description: a11_situacao - Situação do setor do domicílio
Total responses: 1960
Value distribution:
  • Urbano: 1922 (98.06%)
  • Rural: 38 (1.94%)
------------------------------------------------------------

Variable: b02_sexo
Description: b02 - Sexo da criança 
Total responses: 1960
Value distribution:
  • Feminino: 1005 (51.28%)
  • Masculino: 955 (48.72%)
------------------------------------------------------------

Variable: b03_relacao
Description: b03_relacao - Relação da criança com o responsável pelo domicílio
Total responses: 1960
Value 

/opt/anaconda3/envs/pycaret_env/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
